In [41]:
# Import Libraries
from arango import ArangoClient
from arango.exceptions import ArangoError

In [42]:
def validate_graph(db_name="graph_db", graph_name="ai_incidents_graph"):
    """
    Validate the graph structure in ArangoDB by running sample queries.
    """
    try:
        client = ArangoClient(hosts="http://localhost:8529")
        db = client.db(db_name, username="root", password="passwd")

        if not db.has_graph(graph_name):
            print(f"Graph '{graph_name}' does not exist.")
            return

        graph_collection = db.graph(graph_name)
        nodes_collection = db.collection("nodes")
        edges_collection = db.collection("edges")

        node_count = nodes_collection.count()
        edge_count = edges_collection.count()

        print(f"Number of nodes: {node_count}")
        print(f"Number of edges: {edge_count}")

        print("\nFirst 5 nodes:")
        for node in nodes_collection.all(limit=5):
            print(node)

        print("\nFirst 5 edges:")
        for edge in edges_collection.all(limit=5):
            print(edge)

        sample_nodes = list(nodes_collection.all(limit=1))
        if sample_nodes:
            sample_node = sample_nodes[0]["_key"]
            print(f"\nNeighbors of node '{sample_node}':")
            neighbors = db.aql.execute(
                f"FOR v IN 1..1 ANY 'nodes/{sample_node}' edges RETURN v"
            )
            for neighbor in neighbors:
                print(neighbor)
        else:
            print("No sample node found to test neighbor queries.")

    except ArangoError as e:
        print(f"ArangoDB Error: {e}")
    except Exception as e:
        print(f"Unexpected Error: {e}")

# Run validation
validate_graph()


Number of nodes: 487
Number of edges: 0

First 5 nodes:
{'_key': 'Content_Recommendation,_Content_Search,_Hate_Speech_Detection,_NSFW_Content_Detection', '_id': 'nodes/Content_Recommendation,_Content_Search,_Hate_Speech_Detection,_NSFW_Content_Detection', '_rev': '_jO9ChmS---'}
{'_key': 'Content-based_Filtering,_Collaborative_Filtering', '_id': 'nodes/Content-based_Filtering,_Collaborative_Filtering', '_rev': '_jO9Chma---'}
{'_key': 'Hate_Speech_Detection', '_id': 'nodes/Hate_Speech_Detection', '_rev': '_jO9Chme---'}
{'_key': 'Character_NGrams', '_id': 'nodes/Character_NGrams', '_rev': '_jO9Chmm---'}
{'_key': 'Autonomous_Driving', '_id': 'nodes/Autonomous_Driving', '_rev': '_jO9Chmu---'}

First 5 edges:

Neighbors of node 'Content_Recommendation,_Content_Search,_Hate_Speech_Detection,_NSFW_Content_Detection':
